In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import log
from scipy.stats import t
import os
from google.colab import files

In [2]:
def mainMenu():
      
      print(""                 '\n')

            

      print(""                 '\n')
      print(" Software for fitting Photosynthetic Light-Response Curve in aquatic environment ")
      print("                                   by")
      print("                      Arthur Lins"," &  Paulo Lins")
      print(          "Universidade Federal Fluminense (UFF)- RJ, Brazil, arthurlins@id.uff.br ")
      print(          "Secretaria de Estado de Educação do Rio de Janeiro (SEDUC)- RJ, Brazil,  plins@prof.educacao.rj.gov.br" )
      print(""                 '\n') 
      print(" 1.Import data from a spreadsheet  (.csv separated by commas).")
      print(" 2.Enter data")
      print(" 3.Reference and Photosynthetic Parameters ")
      print(" 4.Results")
      
      
      print("Choose the option:")
      
      selection=int(input())
      if selection==2:
                        digitar()
                        os.system("pause")
                        mainMenu()

      elif selection ==1:
                        importar()
                        os.system("pause")
                        mainMenu()

      elif selection ==3:
                        rev()
                        os.system("pause")
                        mainMenu()
      elif selection ==4:
                        results()
                        os.system("pause")
                        mainMenu()
      else:
                        print("Invalid choice. choice enters 1 e 4")
                        mainMenu()

In [3]:
#Funções
def calculate_aic(n, mse, k):
    aic = n * log(mse/n) + 2 * k
    return aic
def func_tan(E,p,a,r):
    return p*(np.tanh((a*E)/p))+r
def func_exp(E,p,a,r):
    return p*(1-np.exp(-a*E/p))+r
def func_exp_inib(E,p,a,b,r):
    return  (p*((1-np.exp(-a*E/p))*np.exp(-b*E/p)))+r
def func_poli_inib(E,a,b,c,r):
    return (E/((a*E**2)+(b*E+c)))+r


In [4]:
def rev ():
  print("\n")
  print(" References:")
  print("\n")
  print("Burnham, K. P., and D. R. Anderson. 2002. Model selection \n and multimodel inference: a practical information theoretic \napproach. Second edition. Springer, New York.\n")

  print("Eilers, P. H. C., and J. C. H. Peeters. 1988. A model for the relationship \n between light intensity and the rate of photosynthesis\n in phytoplankton. Ecol. Model. 42:199-215\n")

  print("Jassby AD, Platt T (1976) Mathematical formulation of the\n relationship between photosynthesis and light for phytoplankton.\n Limnol Oceanogr 21:540–547\n")

  print("Platt, T., C. L. Gallegos, and W. G. Harrison. 1980. Photoinhibition\n and photosynthesis in natural assemblages of\n marine phytoplankton. J. Mar. Res. 38:687-701.\n")

  print("Webb, W. L., M. Newton, and D. Starr. 1974. Carbon dioxide\n exchange of Alnus rubra: A mathematical model. Oecologia\n 17:281-291\n")

  print("Zar, J.H. 2010. Biostatistical analysis. New Jersey, Prentice Hall")
 
  print("\n")
  print("\n")
  print("Photosynthetic Parameters:")
  print("\n")
  print("Pmax primary production (maximum) at saturating light")
  print("\n")
  print("alfa initial slope of PxI curve")
  print("\n")
  print("Ek   adaptation parameter of the PxI curve =Pmax/alfa")
  print("\n")
  print("Rd   dark respiration")
  print("\n")
  print("Ec   light compensation point = Rd/alfa")
  print("\n")
  print("β    photoinhibition index")

In [5]:
def results ():
  print("\n")
  print("Graphics and table results will be in the folder of google colab and you can visualize and download.")
  print("This folder is in the left bar. There's a folder icon. It's in content --> sample_data.")

In [6]:
def importar():
    global lista
    global n
    global xmean
    global xstd
    global ydata
    global xdata
    global Pj
    global Pj1
    global Pj2
    global Pj3
    global ig
    global ig2
    global ig3
    global y
    global n1
    Pj=3
    Pj1=3
    Pj2=4
    Pj3=4
    newlist = []
    
    print(""'\n')

    
    
    uploaded = files.upload()
    print("Insert the independent variable")
    lydigi=input("")
    ly=lydigi
    print("Insert the dependent variable")
    lxdigi=input("")
    lx=lxdigi
    
    print(""'\n')

    


    for i,item in enumerate(uploaded):
        if item.endswith(".csv"):
            newlist.append(item)
            dft = pd.read_csv(item ,sep=";", decimal=",", encoding='latin-1', skiprows=None, header=None, index_col=None)
            print(item)  
             

        df1 = dft.apply(lambda x: pd.Series(x.dropna().values)).dropna(1)
        xmean=df1.iloc[:,1:].mean(axis=1).to_numpy()
        xstd=df1.iloc[:,1:].std(axis=1).fillna(0).to_numpy()
        
        ydata=df1.iloc[:,0].to_numpy()

       
        print("Spreadsheet currently read:" + item)


        xdata=xmean

        ig=[max(xdata),0,xdata[0]]
        ig2=[1,0,0,xdata[0]]
        ig3=[0,0,1,1]

        y=np.linspace(ydata[0],ydata[len(ydata)-1])
        n=len(xdata)
        print("Mean Y values:", xmean)
        print("X values:", ydata)
        print(""                 '\n')
        if np.prod(xstd)>0:
            fit= curve_fit(func_tan, ydata, xdata,sigma=xstd , p0=ig, absolute_sigma=True)
            fit1= curve_fit(func_exp, ydata, xdata,sigma=xstd, p0=ig,absolute_sigma=True)
            fit2= curve_fit(func_exp_inib, ydata, xdata,sigma=xstd, p0=ig2,absolute_sigma=True)
            fit3= curve_fit(func_poli_inib, ydata, xdata,sigma=xstd, p0=ig3,maxfev=5000,absolute_sigma=True)

            ans,cov=fit
            ans1,cov1=fit1
            ans2,cov2=fit2
            ans3,cov3=fit3
            perr = np.sqrt(np.diag(cov))
            perr1 = np.sqrt(np.diag(cov1))
            perr2 = np.sqrt(np.diag(cov2))
            perr3 = np.sqrt(np.diag(cov3))

            fit_p,fit_a,fit_r = ans
            fit_p1,fit_a1,fit_r1 = ans1
            fit_p2,fit_a2,fit_b2,fit_r2 = ans2
            fit_a3,fit_b3,fit_c3,fit_r3= ans3



            plt.figure(figsize=(6, 3))
            plt.grid(True)





            plt.xlabel(ly)

            plt.ylabel(lx)

            plt.scatter(ydata,xdata)
            plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')

            plt.savefig(f"/content/"+ item.split('.')[0]+"points.png")
            plt.show()
        else:
            fit= curve_fit(func_tan, ydata, xdata , p0=ig)
            fit1= curve_fit(func_exp, ydata, xdata, p0=ig)
            fit2= curve_fit(func_exp_inib, ydata, xdata, p0=ig2)
            fit3= curve_fit(func_poli_inib, ydata, xdata, p0=ig3,maxfev=5000)

            ans,cov=fit
            ans1,cov1=fit1
            ans2,cov2=fit2
            ans3,cov3=fit3
            perr = np.sqrt(np.diag(cov))
            perr1 = np.sqrt(np.diag(cov1))
            perr2 = np.sqrt(np.diag(cov2))
            perr3 = np.sqrt(np.diag(cov3))

            fit_p,fit_a,fit_r = ans
            fit_p1,fit_a1,fit_r1 = ans1
            fit_p2,fit_a2,fit_b2,fit_r2 = ans2
            fit_a3,fit_b3,fit_c3,fit_r3= ans3



            plt.figure(figsize=(6, 4))
            plt.grid(True)

            plt.xlabel(ly)

            plt.ylabel(lx)

            plt.scatter(ydata,xdata)
            plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')

            plt.savefig(f"/content/"+ item.split('.')[0]+"points.png")
            plt.show()

            
        

        ftr1=np.empty((len(ydata)))
        ftr1[:] = np.NaN

    #Func
        ftr=np.empty((len(ydata)))
        ftr[:] = np.NaN

        i=0
        for ydatar in ydata:
            ftr[i]=func_tan(ydatar,ans[0],ans[1],ans[2])
            i=i+1

    #Func1
        ftr1=np.empty((len(ydata)))
        ftr1[:] = np.NaN

        i=0
        for ydatar in ydata:
            ftr1[i]=func_exp(ydatar,ans1[0],ans1[1],ans1[2])
            i=i+1

        #Func2
        ftr2=np.empty((len(ydata)))
        ftr2[:] = np.NaN

        i=0
        for ydatar in ydata:
            ftr2[i]=func_exp_inib(ydatar,ans2[0],ans2[1],ans2[2],ans2[3])
            i=i+1

        #Func3
        ftr3=np.empty((len(ydata)))


        i=0
        for ydatar in ydata:
            ftr3[i]=func_poli_inib(ydatar,ans3[0],ans3[1],ans3[2],ans3[3])
            i=i+1




        
        print("1. Adjustment of curves without photo-inhibition( Tangent hyperbolic, Jassby and Platt,1976 ; \nExponential, Webb et al. 1974 ; Rational de Eilers and Peeters, 1988 )")
        print("2. Adjusting curves with photo-inhibition ( Double Exponential , Platt et al. 1980  ; \nRational de Eilers and Peeters, 1988)")
        a=int(input())
        print(""                    '\n')


        if a==1:

                print("Results:" + item)
                func_tanh=print('Tangent hyperbolic:'),print('P=Pmax*(tanh(α*E/Pmax)+Rd'),print('P='+str(round(ans[0],2))+'*'+'tanh(('+ str(round(ans[1],3))+'*E/'+str(round(ans[0],2))+')'+ str(round(ans[2],2)))
                print("\n")
                print("Data means and Standard errors")
                print("\n")
                print('Pmax:',round(ans[0],2),"±",round(perr[0],3),'| ''α:', round(ans[1],5),"±",round(perr[1],5),'| ''Rd:', round(ans[2],5),"±",round(perr[2],3))



                ept=((perr[0]**2)+(perr[2]**2))**(0.5)
                pt=ans[0]+ans[2]
                ek= (ans[0])/ans[1]
                erroek=(ept/pt)*(ek)*(1/2)

                print("Ek:", round(ek,2),"±",round(erroek,2))
                ec=-(ans[2]/ans[1])
                erroec=(((((perr[2]/ans[2])**2))+((perr[1]/ans[1])**2))*(ec**2))**(1/2)
                print("Ec:", round(ec,2),"±",round(erroec,2))
                
                
                r2=r2_score(xdata,ftr, sample_weight=None, multioutput='uniform_average')
                r2adj=1-(1-r2)*(n-1)/(n-(Pj-1))
                rss=((xdata-ftr)**2).sum()
                print("R squared:", round(r2,3))
                print("Ajusted R squared:", round(r2adj,3))
                mse = mean_squared_error(xdata, ftr)
                aic = calculate_aic(n, rss, Pj)
                aiccc=aic+2*(Pj*(Pj)/(n-Pj-1))
                                    
                sxy=((((xdata-ftr)**2).sum())/(n-3))**(0.5)
                print("Sxy:", round(sxy,2))
                print("AICc:", round(aiccc,2)) 


                plt.figure(figsize=(6, 4))
                plt.grid(True)

                plt.plot(y,func_tan(y,fit_p,fit_a,fit_r),label= 'P=Pmax*(tanh(α*E/p)+Rd' ) 
                plt.legend()
                plt.xlabel(ly)
                plt.ylabel(lx)
                plt.scatter(ydata,xdata)
                plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
                print("The program will continue to process after the graph is closed, it was saved in the same folder as the data file")
                plt.savefig(f"/content/"+ item.split('.')[0]+"tangent.function.png")
                
                
                plt.show()
                

                print(""                    '\n')
  
                
                print("Results:" + item)
                func_expe = print('Exponential:'),print('P=Pmax*(1-e^(-α*E/Pmax))+Rd'),print('P='+str(round(ans1[0],2))+'*'+'(1-e^(-'+ str(round(ans1[1],3))+'*E/'+str(round(ans1[0],2))+')'+ str(round(ans1[2],2)))
                print("\n")
                print("Data means and Standard errors")
                print("\n")
                print('Pmax:',round(ans1[0],2),"±",round(perr1[0],3),'| ''α:', round(ans1[1],4),"±",round(perr1[1],3),'| ''Rd:', round(ans1[2],2),"±",round(perr1[2],3))
                


                ept1=((perr1[0]**2)+(perr1[2]**2))**(0.5)
                pt1=ans1[0]+ans1[2]
                ek1= (ans1[0])/ans1[1]
                erroek1=(ept1/pt1)*(ek1)*(1/2)

                print("Ek:", round(ek1,2),"±", round(erroek1,2))
                ec1=-(ans1[2]/ans1[1])
                erroec1=(((((perr1[2]/ans1[2])**2))+((perr1[1]/ans1[1])**2))*(ec1**2))**(1/2)
                print("Ec:", round(ec1,2),"±",round(erroec1,2))
                
                
                r21=r2_score(xdata,ftr1, sample_weight=None, multioutput='uniform_average')
                r2adj1=1-(1-r21)*(n-1)/(n-(Pj1-1))
                rss1=((xdata-ftr1)**2).sum()
                print("R squared:", round(r21,3))
                print("Ajusted R squared:", round(r2adj1,3))
                mse1 = mean_squared_error(xdata, ftr1)
                aic1 = calculate_aic(n, rss1, Pj1)
                aiccc1=aic1+(2*(Pj1*(Pj1)/(n-Pj1-1)))
                
                sxy1=((((ftr1-xdata)**2).sum())/(n-3))**(0.5)
                print("Sxy:", round(sxy1,2))
                print("AICc:", round(aiccc1,2))
                
                

                plt.figure(figsize=(6, 4))
                plt.grid(True)

                plt.plot(y,func_exp(y,fit_p1,fit_a1,fit_r1),label='P=Pmax*(1-e^(-α*E/p))+Rd') 
                plt.legend()
                plt.xlabel(ly)
                plt.ylabel(lx)
                plt.scatter(ydata,xdata)
                plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
                plt.savefig(f"/content/"+ item.split('.')[0]+"exponential.function.png")
                print("\n")

                plt.show()
                

                print(""                    '\n')

                print("Results:" + item)
                func_poli_inibi=print('Rational:'),print('P=(E/((a*E^2))+(b*E+c)))+Rd'),print('Ps= (E/(('+str(round(fit_a3,7))+'*E²)+('+ str(round(fit_b3,7))+'*E+'+ str(round(fit_c3,2)) +'))+('+ str(round(fit_r3,2))+')')
                print("\n")
                print("Data means and Standard errors")
                print("\n")
                print('a:',round(ans3[0],7),"±",round(perr3[0],7),'| ''b:', round(ans3[1],10),"±",round(perr3[1],3),'| ''c:', round(ans3[2],2),"±",round(perr3[2],3),'| ''Rd:', round(ans3[3],5),"±",round(perr3[3],3))
  
                pmax3= (1/(ans3[1]+2*((ans3[0]*ans3[2])**(1/2)))+ans3[3])
                s31=ans3[0]*ans3[2]
                serro31=((((perr3[0]/ans3[0])**2)+((perr3[2]/ans3[2])**2))*(s31**2))**(0.5)
                s311=s31**(0.5)
                erros311=(1/2)*(serro31/s31)*s311
                s312=2*s311
                erros312=((((erros311/s311)**2)+((0/ans3[2])**2))*(s312**2))**(0.5)
                erros313=((perr3[1]**2)+(erros312**2))**(0.5)
                s314=1/(s312+ans3[1])
                erros314=((((erros312/s312)**2)+((0/ans3[2])**2))*(s314**2))**(0.5)
                s315=s314+ans3[3]
                erros315=((perr3[3]**2)+(erros314**2))**(0.5)

                print("Pmax:", round(pmax3,2),"±",round(erros315,2))
                alfa=1/ans3[2]
                erroalfa3=((((perr3[2]/ans3[2])**2)+((0/ans3[2])**2))*(alfa**2))**(0.5)
                print("α:", round(alfa,2),"±",round(erroalfa3,2))

                ept3=((erros315**2)+(perr3[3]**2))**(0.5)
                pt3=pmax3+ans3[3]
                ek3= (pmax3)/alfa
                erroek3=(ept3/pt3)*(ek3)*(1/2)
                print("Ek:", round(ek3,2),"±",round(erroek3,2))
                ec3=-(ans3[3]/alfa)
                erroec3=(((((perr3[3]/ans3[3])**2))+((erroalfa3/alfa)**2))*(ec3**2))**(1/2)
                print("Ec:", round(ec3,2),"±",round(erroec3,2))
                bb3=(ans3[1])/(((ans3[0]*ans3[2])**(1/2)))
                errobbpt1=((((perr3[0]/ans3[0])**2)+((perr3[2]/ans3[2])**2))*((ans3[0]*ans3[2])**2))**(1/2)
                errobbpt2=errobbpt1*(1/2)*(((ans3[0]*ans3[2])**(1/2)))
                errobb=(erroalfa3/10)
                print("β:",round(bb3,2),"±",round(errobb,7))
                
                r23=r2_score(xdata,ftr3, sample_weight=None, multioutput='uniform_average')
                r2adj3=1-(1-r23)*(n-1)/(n-(Pj3-1) )
                rss3=((xdata-ftr3)**2).sum()
                print("R squared:", round(r23,3))
                print("Ajusted R squared:", round(r2adj3,3))
                mse3 = mean_squared_error(xdata, ftr3)
                aic3 = calculate_aic(n, rss3, Pj3)
                aiccc3=aic3+2*(Pj3*(Pj3)/(n-(Pj3)-1))
                
                sxy3=((((ftr3-xdata)**2).sum())/(n-4))**(0.5)
                print("Sxy:", round(sxy3,2))
                print("AICc:", round(aiccc3,2))
                
                rt=(ans[2])
                rt1=(ans1[2])
                rt3=(ans3[3])
                alfat=(ans[1])
                alfat1=(ans1[1])
                alfat3=(alfa)

                plt.figure(figsize=(6, 4))
                plt.grid(True)

                plt.plot(y,func_poli_inib(y,fit_a3,fit_b3,fit_c3,fit_r3), label='P=(E/((a*E**2)+(b*E+c)))+Rd') 
                plt.legend()
                plt.xlabel(ly)
                plt.ylabel(lx)
                plt.scatter(ydata,xdata)
                plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
                plt.savefig(f"/content/"+ item.split('.')[0]+"Rational.function.png")
                print("The program will continue to process after the graph is closed, it was saved in the same folder as the data file")
                plt.show()
                
                
                print(""                    '\n')
   
                
                
                
                



                        
          
                
                df2 = pd.DataFrame({'Model :':'tangent hyperbolic (P=Pmax*(tanh(alfa*E/Pmax)+Rd)' ,
                                    'Pmax':round(ans[0],2) ,
                                    'SE-1': (round(perr[0],2)),
                                    'alfa':round(alfat,2) ,
                                    'SE-2': (round(perr[1],2)),
                                    'Ek':round(ek,2),
                                    'SE-3': (round(erroek,2)),
                                    'Ec':round(ec,2),
                                    'SE-4': (round(erroec,2)),
                                    'beta': ("-"),
                                    'SE-5': ("-"),
                                    'Rd': round(rt,2),
                                    'SE-6': (round(perr[2],2)),
                                    'R squared':round(r2,3),
                                    'Ajusted R squared':round(r2adj,3),
                                    'Sxy':round(sxy,2),
                                    'AICc':round(aiccc,2),
                                    },index= [item])
                df3 = pd.DataFrame({'Model :':'exponential (P=Pmax*(1-e^(-alfa*E/Pmax))+Rd)' ,
                                    'Pmax':round(ans1[0],2) ,
                                    'SE-1': (round(perr1[0],2)),
                                    'alfa':round(alfat1,2) ,

                                    'SE-2': (round(perr1[1],2)),
                                    'Ek':round(ek1,2),
                                    'SE-3': (round(erroek1,2)),
                                    'Ec':round(ec1,2),
                                    'SE-4': (round(erroec1,2)),
                                    'beta': ("-"),
                                    'SE-5': ("-"),
                                    'Rd': round(rt1,2),
                                    'SE-6': (round(perr1[2],2)),
                                    'R squared':round(r21,3),
                                    'Ajusted R squared':round(r2adj1,3),
                                    'Sxy':round(sxy1,2),
                                    'AICc':round(aiccc1,2),
                                    },index= [item])
                df5 = pd.DataFrame({'Model :':'rational (P=(E/((a*E^2))+(b*E+c)))+Rd)'  ,
                                    'Pmax':round(pmax3,2) ,
                                    'SE-1': (round(erros315,2)),
                                    'alfa':round(alfat3,2) ,
                                    'SE-2': (round(erroalfa3,2)),
                                    'Ek':round(ek3,2),
                                    'SE-3': (round(erroek3,2)),
                                    'Ec':round(ec3,2),
                                    'SE-4': (round(erroec3,2)),
                                    'beta':(round(bb3,7)),
                                    'SE-5': (round(errobb,7)),
                                    'Rd': round(rt3,2),
                                    'SE-6': (round(perr3[3],2)),
                                    'R squared': (round(r23,3)),
                                    
                                    'Ajusted R squared':round(r2adj3,3),
                                    'Sxy':round(sxy3,2),
                                    'AICc':round(aiccc3,2),
                                     },index= [item])
                frames=[df2,df3,df5]
                results = pd.concat(frames)
                results.to_csv(f"/content/" + "results"+item,sep=',')
                print("File containing the results was created in your folder content at google colab")
                print(""                     '\n')
                print(""                     '\n')
        else:
                print("Results:" + item)
                func_exp_inibi=print('Exponential function for cases of inhibition:'),print('P=(Ps*((1-e^(-α*E/Ps))*e^(-ß*E/Ps)))+Rd'),print('Ps='+str(round(fit_p2,2))+'*'+'((1-e^('+ '-' + str(round(fit_a2,3))+'*E/'+str(round(fit_p2,2))+'))*e^(-'+ str(round(fit_b2,5)) +'*E/'+ str(round(fit_p2,2))+'))'+ str(round(fit_r1,2)))
                print("\n")
                print("Data means and Standard errors")
                print("\n")                
                print('Ps:',round(ans2[0],2),"±",round(perr2[0],3),'| ''α:', round(ans2[1],4),"±",round(perr2[1],3),'| ''ß:', round(ans2[2],7),"±",round(perr2[2],3),'| ''Rd:', round(ans2[3],2),"±",round(perr2[3],3))
               


                pmax2= (ans2[0])*(ans2[1]/(ans2[1]+ans2[2]))*((ans2[2]/(ans2[1]+ans2[2]))**(ans2[2]/ans2[1]))+ans2[3]

                somaerro=ans2[1]+ans2[2]
                errosoma3=(((perr2[1])**2) + ((perr2[2])**2))**(0.5)
                somaa32 = ans2[1]/somaerro
                errosoma32=((((somaerro/errosoma3)**2)+((perr2[1]/ans2[1])**2))*(somaa32**2))**(0.5)
                soma322=ans2[2]/somaerro
                errosoma322=((((somaerro/errosoma3)**2)+((perr2[2]/ans2[2])**2))*(soma322**2))**(0.5)
                expoente1=(ans2[2]/ans2[1])
                erroexpoente1=((((perr2[1]/ans2[1])**2)+((perr2[2]/ans2[2])**2))*(expoente1**2))**(0.5)
                expoente2=soma322**expoente1
                erroexpoente2=(expoente1*(errosoma322/soma322))*expoente2
                multi1=somaa32*expoente2
                erromulti1=((((erroexpoente2/expoente2)**2)+((errosoma32/somaa32)**2))*(multi1**2))**(0.5)
                multi2=multi1*ans2[0]
                erromulti2=((((perr2[0]/ans2[0])**2)+((erromulti1/multi1)**2))*(multi2**2))**(0.5)
                somaf=multi2+ans2[3]
                errosomaf=(erromulti2+perr2[3])**(0.5)
                print("Pmax:", round(pmax2,2),"±",round(errosomaf,2))
                print('α:', round(ans2[1],4),"±",round(perr2[1],3))
                print('ß:', round(ans2[2],7),"±",round(perr2[2],3))
                
                r22=r2_score(xdata,ftr2, sample_weight=None, multioutput='uniform_average')
                r2adj2=1-((1-r22)*(n-1)/(n-(Pj2-1)))
                rss2=((xdata-ftr2)**2).sum()
                ept2=((errosomaf**2)+(perr2[3]**2))**(0.5)
                pt2=pmax2+ans2[3]
                ek2= (pmax2)/ans2[1]
                erroek2=(ept2/pt2)*(ek2)*(1/2)
                print("Ek:", round(ek2,2),"±",round(erroek2,2))
                ec2=-(ans2[3]/ans2[1])
                erroec2=(((((perr2[3]/ans2[3])**2))+((perr2[1]/ans2[1])**2))*(ec2**2))**(1/2)
                print("Ec:", round(ec2,2),"±",round(erroec2,2))
                print("R squared:", round(r22,3))
                print("Ajusted R squared:", round(r2adj2,3))
                mse2 = mean_squared_error(xdata, ftr2)
                aic2 = calculate_aic(n, rss2, Pj2)
                aiccc2=aic2+2*(Pj2*(Pj2)/(n-Pj2-1))
                
                sxy2=((((ftr2-xdata)**2).sum())/(n-4))**(0.5)
                print("Sxy:", round(sxy2,2))
                print("AICc:", round(aiccc2,2))




   

                plt.figure(figsize=(6, 4))
                plt.grid(True)

                plt.plot(y,func_exp_inib(y,fit_p2,fit_a2,fit_b2,fit_r2),label = 'P=Ps*(1-np.exp(-α*E/P)*np.exp(-ß*E/P))+Rd') 
                plt.legend()
                plt.xlabel(ly)
                plt.ylabel(lx)
                plt.scatter(ydata,xdata)
                plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
                plt.savefig(f"/content/"+ item.split('.')[0]+"exponential.double.function.png")
                print("\n")
                plt.show()
                

                print(""                     '\n')

                print("Results:" + item)
                func_poli_inibi=print('Rational  Function:'),print('P=(E/((a*E^2))+(b*E+c)))+Rd'),print('Ps= (E/(('+str(round(fit_a3,7))+'*E²)+('+ str(round(fit_b3,7))+'*E+'+ str(round(fit_c3,2)) +'))+('+ str(round(fit_r3,2))+')')
                print("\n")
                print("Data means and Standard errors")
                print("\n")
                print('a:',round(ans3[0],7),"±",round(perr3[0],7),'| ''b:', round(ans3[1],10),"±",round(perr3[1],3),'| ''c:', round(ans3[2],2),"±",round(perr3[2],3),'| ''Rd:', round(ans3[3],5),"±",round(perr3[3],3))
                
                
                pmax3= (1/(ans3[1]+2*((ans3[0]*ans3[2])**(1/2)))+ans3[3])
                s31=ans3[0]*ans3[2]
                serro31=((((perr3[0]/ans3[0])**2)+((perr3[2]/ans3[2])**2))*(s31**2))**(0.5)
                s311=s31**(0.5)
                erros311=(1/2)*(serro31/s31)*s311
                s312=2*s311
                erros312=((((erros311/s311)**2)+((0/ans3[2])**2))*(s312**2))**(0.5)
                erros313=((perr3[1]**2)+(erros312**2))**(0.5)
                s314=1/(s312+ans3[1])
                erros314=((((erros312/s312)**2)+((0/ans3[2])**2))*(s314**2))**(0.5)
                s315=s314+ans3[3]
                erros315=((perr3[3]**2)+(erros314**2))**(0.5)
                print("Pmax:", round(pmax3,2),"±",round(erros315,2))
                alfa=1/ans3[2]
                erroalfa3=((((perr3[2]/ans3[2])**2)+((0/ans3[2])**2))*(alfa**2))**(0.5)
                print("α:", round(alfa,2),"±",round(erroalfa3,2))
                ept3=((erros315**2)+(perr3[3]**2))**(0.5)
                pt3=pmax3+ans3[3]
                ek3= (pmax3)/alfa
                erroek3=(ept3/pt3)*(ek3)*(1/2)
                print("Ek:", round(ek3,2),"±",round(erroek3,2))
                ec3=-(ans3[3]/alfa)
                erroec3=(((((perr3[3]/ans3[3])**2))+((erroalfa3/alfa)**2))*(ec3**2))**(1/2)
                print("Ec:", round(ec3,2),"±",round(erroec3,2))
                bb3=(ans3[1])/(((ans3[0]*ans3[2])**(1/2)))
                errobbpt1=((((perr3[0]/ans3[0])**2)+(perr3[2]/ans3[2])**2)*((ans3[0]*ans3[2])**2))**(1/2)
                errobbpt2=errobbpt1*(1/2)*(((ans3[0]*ans3[2])**1/2))
                errobb=erroalfa3/10
                print("β:",round(bb3,2),"±",round(errobb,7))
                
                r23=r2_score(xdata,ftr3, sample_weight=None, multioutput='uniform_average')
                r2adj3=1-(1-r23)*(n-1)/(n-(Pj3-1) )
                rss3=((xdata-ftr3)**2).sum()
                print("R squared:", round(r23,3))
                print("Ajusted R squared:", round(r2adj3,3))
                mse3 = mean_squared_error(xdata, ftr3)
                aic3 = calculate_aic(n, rss3, Pj3)
                aiccc3=aic3+2*(Pj3*(Pj3)/(n-(Pj3)-1))
                
                sxy3=((((ftr3-xdata)**2).sum())/(n-4))**(0.5)
                print("Sxy:", round(sxy3,2))
                print("AICc:", round(aiccc3,2))

                plt.figure(figsize=(6, 4))
                plt.grid(True)

                plt.plot(y,func_poli_inib(y,fit_a3,fit_b3,fit_c3,fit_r3), label='P=(E/((a*E**2)+(b*E+c)))+Rd') 
                plt.legend()
                plt.xlabel(ly)
                plt.ylabel(lx)
                plt.scatter(ydata,xdata)
                plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
                plt.savefig(f"/content/"+ item.split('.')[0]+"rational function.png")
                print("\n")
                plt.show()
                
                
                
                rt2=(ans2[3])
                rt3=(ans3[3])
               
                alfat2=(ans2[1])
                alfat3=(alfa)
                pmaxt2=(ans2[0])
                epmaxt2=(perr2[0])
                
                
                df4 = pd.DataFrame({'Model :':'double exponential (P=(Ps*((1-e^(-alfa*E/Ps))*e^(-ß*E/Ps)))+Rd)'  ,
                                            'Pmax':round(pmax2,2) ,
                                            'SE-1': (round(errosomaf,2)),
                                            'alfa':round(alfat2,2) ,
                                            'SE-2': (round(perr2[1],2)),
                                            'Ek':round(ek2,2),
                                            'SE-3': (round(erroek2,2)),
                                            'Ec':round(ec2,2),
                                            'SE-4': (round(erroec2,2)),
                                            'beta':(round(ans2[2],7)),
                                            'SE-5': (round(perr2[2],7)),
                                            'Rd':round(rt2,2),
                                            'SE-6': (round(perr2[3],2)),
                                            'R squared':round(r22,3),
                                            'Ajusted R squared':round(r2adj2,3),
                                            'Sxy':round(sxy2,2),
                                            'AICc':round(aiccc2,2),
                      },index= [item])
                df5 = pd.DataFrame({'Model :':'rational (P=(E/((a*E^2))+(b*E+c)))+Rd)'  ,
                                    'Pmax':round(pmax3,2) ,
                                    'SE-1': (round(erros315,2)),
                                    'alfa':round(alfat3,2) ,
                                    'SE-2': (round(erroalfa3,2)),
                                    'Ek':round(ek3,2),
                                    'SE-3': (round(erroek3,2)),
                                    'Ec':round(ec3,2),
                                    'SE-4': (round(erroec3,2)),
                                    'beta':(round(bb3,2)),
                                    'SE-5': (round(errobb,2)),
                                    'Rd': round(rt3,2),
                                    'SE-6': (round(perr3[3],2)),
                                    'R squared':round(r23,3),
                                    'Ajusted R squared':round(r2adj3,3),
                                    'Sxy':round(sxy3,2),
                                    'AICc':round(aiccc3,2),
                     },index= [item])
    



     


       

                frames=[df4,df5]
                results = pd.concat(frames)
                results.to_csv(f"/content/" + "results"+item,sep=',')
                print("File containing the results was created in your folder content at google colab")
                print(""                    '\n')
                print(""                    '\n')

In [7]:
def digitar():
    global lista
    global n
    global xmean
    global xstd
    global ydata
    global xdata
    global Pj
    global Pj1
    global Pj2
    global Pj3
    global ig
    global ig2
    global ig3
    global y
    global n1
    global ly
    global lx
    Pj=3
    Pj1=3
    Pj2=4
    Pj3=4
    n=int(input("Enter the sample size, that is, the number of X-axis data: "))
    lista2=["","","","","","","","","","","","","","","","","","","","","","","","","",""]
    lista=["","","","","","","","","","","","","","","","","","","","","","","","","",""]
    print("Enter the values ​​of the Y axis, either a value or its replicas side by side with the decimal separator being '.' and the data separator being ',', and then enter the axis values x ")
    for i in range (0,n):
        
        aa=str(input())
        lista[i]=np.fromstring(aa, dtype=float, count=-1, sep=',')
    i=0
    
    bb=str(input())
    ydata=np.fromstring(bb, dtype=float, count=-1, sep=',')
    
    
    xmean=np.empty((9))
    xmean[:] = np.NaN
    xstd=np.empty((9))
    xstd[:] = np.NaN
    print("Insert the independent variable")
    lydigi=input("")
    ly=lydigi
    print("Insert the dependent variable")
    lxdigi=input("")
    lx=lxdigi
    
    print(""'\n')
    for i in range(0,9):
        if len(lista[i])>0:
            xmean[i]=lista[i].mean()
            xstd[i]=lista[i].std()
    xmean = xmean[~np.isnan(xmean)]
    xstd=xstd[~np.isnan(xstd)]
    
    xdata=xmean
    ig=[max(xdata),0,xdata[0]]
    ig2=[1,0,0,xdata[0]]
    ig3=[0,0,1,1]

    y=np.linspace(ydata[0],ydata[len(ydata)-1])
    n=len(xdata)
    print(""
                 '\n')

    print("Average values ​​of Y:", xmean)
    print("Average of X:", ydata)
    print(""
                 '\n')

    
    if np.prod(xstd)>0:
        fit= curve_fit(func_tan, ydata, xdata,sigma=xstd , p0=ig, absolute_sigma=True)
        fit1= curve_fit(func_exp, ydata, xdata,sigma=xstd, p0=ig,absolute_sigma=True)
        fit2= curve_fit(func_exp_inib, ydata, xdata,sigma=xstd, p0=ig2,absolute_sigma=True)
        fit3= curve_fit(func_poli_inib, ydata, xdata,sigma=xstd, p0=ig3,maxfev=5000,absolute_sigma=True)

        ans,cov=fit
        ans1,cov1=fit1
        ans2,cov2=fit2
        ans3,cov3=fit3
        perr = np.sqrt(np.diag(cov))
        perr1 = np.sqrt(np.diag(cov1))
        perr2 = np.sqrt(np.diag(cov2))
        perr3 = np.sqrt(np.diag(cov3))

        fit_p,fit_a,fit_r = ans
        fit_p1,fit_a1,fit_r1 = ans1
        fit_p2,fit_a2,fit_b2,fit_r2 = ans2
        fit_a3,fit_b3,fit_c3,fit_r3= ans3



        plt.figure(figsize=(6, 3))
        plt.grid(True)





        plt.xlabel(ly)

        plt.ylabel(lx)

        plt.scatter(ydata,xdata)
        plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')

        plt.savefig(f"/content/"+"points.png")
        plt.show()
    else:
        fit= curve_fit(func_tan, ydata, xdata , p0=ig)
        fit1= curve_fit(func_exp, ydata, xdata, p0=ig)
        fit2= curve_fit(func_exp_inib, ydata, xdata, p0=ig2)
        fit3= curve_fit(func_poli_inib, ydata, xdata, p0=ig3,maxfev=5000)

        ans,cov=fit
        ans1,cov1=fit1
        ans2,cov2=fit2
        ans3,cov3=fit3
        perr = np.sqrt(np.diag(cov))
        perr1 = np.sqrt(np.diag(cov1))
        perr2 = np.sqrt(np.diag(cov2))
        perr3 = np.sqrt(np.diag(cov3))

        fit_p,fit_a,fit_r = ans
        fit_p1,fit_a1,fit_r1 = ans1
        fit_p2,fit_a2,fit_b2,fit_r2 = ans2
        fit_a3,fit_b3,fit_c3,fit_r3= ans3



        plt.figure(figsize=(6, 4))
        plt.grid(True)

        plt.xlabel(ly)

        plt.ylabel(lx)

        plt.scatter(ydata,xdata)
        plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')

        plt.savefig(f"/content/"+ "points.png")
        plt.show()

        
    

    ftr1=np.empty((len(ydata)))
    ftr1[:] = np.NaN

#Func
    ftr=np.empty((len(ydata)))
    ftr[:] = np.NaN

    i=0
    for ydatar in ydata:
        ftr[i]=func_tan(ydatar,ans[0],ans[1],ans[2])
        i=i+1

#Func1
    ftr1=np.empty((len(ydata)))
    ftr1[:] = np.NaN

    i=0
    for ydatar in ydata:
        ftr1[i]=func_exp(ydatar,ans1[0],ans1[1],ans1[2])
        i=i+1

    #Func2
    ftr2=np.empty((len(ydata)))
    ftr2[:] = np.NaN

    i=0
    for ydatar in ydata:
        ftr2[i]=func_exp_inib(ydatar,ans2[0],ans2[1],ans2[2],ans2[3])
        i=i+1

    #Func3
    ftr3=np.empty((len(ydata)))


    i=0
    for ydatar in ydata:
        ftr3[i]=func_poli_inib(ydatar,ans3[0],ans3[1],ans3[2],ans3[3])
        i=i+1




    
    print("1. Adjustment of curves without photo-inhibition( Tangent hyperbolic, Jassby and Platt,1976 ; \nExponential, Webb et al. 1974 ; Rational de Eilers and Peeters, 1988 )")
    print("2. Adjusting curves with photo-inhibition ( Double Exponential , Platt et al. 1980  ; \nRational de Eilers and Peeters, 1988)")
    a=int(input())
    print(""                    '\n')


    if a==1:

            print("Results:" )
            func_tanh=print('Tangent hyperbolic:'),print('P=Pmax*(tanh(α*E/Pmax)+Rd'),print('P='+str(round(ans[0],2))+'*'+'tanh(('+ str(round(ans[1],3))+'*E/'+str(round(ans[0],2))+')'+ str(round(ans[2],2)))
            print("\n")
            print("Data means and Standard errors")
            print("\n")
            print('Pmax:',round(ans[0],2),"±",round(perr[0],3),'| ''α:', round(ans[1],5),"±",round(perr[1],5),'| ''Rd:', round(ans[2],5),"±",round(perr[2],3))



            ept=((perr[0]**2)+(perr[2]**2))**(0.5)
            pt=ans[0]+ans[2]
            ek= (ans[0])/ans[1]
            erroek=(ept/pt)*(ek)*(1/2)

            print("Ek:", round(ek,2),"±",round(erroek,2))
            ec=-(ans[2]/ans[1])
            erroec=(((((perr[2]/ans[2])**2))+((perr[1]/ans[1])**2))*(ec**2))**(1/2)
            print("Ec:", round(ec,2),"±",round(erroec,2))
            
            
            r2=r2_score(xdata,ftr, sample_weight=None, multioutput='uniform_average')
            r2adj=1-(1-r2)*(n-1)/(n-(Pj-1))
            rss=((xdata-ftr)**2).sum()
            print("R squared:", round(r2,3))
            print("Ajusted R squared:", round(r2adj,3))
            mse = mean_squared_error(xdata, ftr)
            aic = calculate_aic(n, rss, Pj)
            aiccc=aic+2*(Pj*(Pj)/(n-Pj-1))
                                
            sxy=((((xdata-ftr)**2).sum())/(n-3))**(0.5)
            print("Sxy:", round(sxy,2))
            print("AICc:", round(aiccc,2)) 


            plt.figure(figsize=(6, 4))
            plt.grid(True)

            plt.plot(y,func_tan(y,fit_p,fit_a,fit_r),label= 'P=Pmax*(tanh(α*E/p)+Rd' ) 
            plt.legend()
            plt.xlabel(ly)
            plt.ylabel(lx)
            plt.scatter(ydata,xdata)
            plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
            print("The program will continue to process after the graph is closed, it was saved in the same folder as the data file")
            plt.savefig(f"/content/"+ "tangent.function.png")
            
            
            plt.show()
            

            print(""                    '\n')

            
            print("Results:" )
            func_expe = print('Exponential:'),print('P=Pmax*(1-e^(-α*E/Pmax))+Rd'),print('P='+str(round(ans1[0],2))+'*'+'(1-e^(-'+ str(round(ans1[1],3))+'*E/'+str(round(ans1[0],2))+')'+ str(round(ans1[2],2)))
            print("\n")
            print("Data means and Standard errors")
            print("\n")
            print('Pmax:',round(ans1[0],2),"±",round(perr1[0],3),'| ''α:', round(ans1[1],4),"±",round(perr1[1],3),'| ''Rd:', round(ans1[2],2),"±",round(perr1[2],3))
            


            ept1=((perr1[0]**2)+(perr1[2]**2))**(0.5)
            pt1=ans1[0]+ans1[2]
            ek1= (ans1[0])/ans1[1]
            erroek1=(ept1/pt1)*(ek1)*(1/2)

            print("Ek:", round(ek1,2),"±", round(erroek1,2))
            ec1=-(ans1[2]/ans1[1])
            erroec1=(((((perr1[2]/ans1[2])**2))+((perr1[1]/ans1[1])**2))*(ec1**2))**(1/2)
            print("Ec:", round(ec1,2),"±",round(erroec1,2))
            
            
            r21=r2_score(xdata,ftr1, sample_weight=None, multioutput='uniform_average')
            r2adj1=1-(1-r21)*(n-1)/(n-(Pj1-1))
            rss1=((xdata-ftr1)**2).sum()
            print("R squared:", round(r21,3))
            print("Ajusted R squared:", round(r2adj1,3))
            mse1 = mean_squared_error(xdata, ftr1)
            aic1 = calculate_aic(n, rss1, Pj1)
            aiccc1=aic1+(2*(Pj1*(Pj1)/(n-Pj1-1)))
            
            sxy1=((((ftr1-xdata)**2).sum())/(n-3))**(0.5)
            print("Sxy:", round(sxy1,2))
            print("AICc:", round(aiccc1,2))
            
            

            plt.figure(figsize=(6, 4))
            plt.grid(True)

            plt.plot(y,func_exp(y,fit_p1,fit_a1,fit_r1),label='P=Pmax*(1-e^(-α*E/p))+Rd') 
            plt.legend()
            plt.xlabel(ly)
            plt.ylabel(lx)
            plt.scatter(ydata,xdata)
            plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
            plt.savefig(f"/content/"+ "exponential.function.png")
            print("\n")

            plt.show()
            

            print(""                    '\n')

            print("Results:" )
            func_poli_inibi=print('Rational:'),print('P=(E/((a*E^2))+(b*E+c)))+Rd'),print('Ps= (E/(('+str(round(fit_a3,7))+'*E²)+('+ str(round(fit_b3,7))+'*E+'+ str(round(fit_c3,2)) +'))+('+ str(round(fit_r3,2))+')')
            print("\n")
            print("Data means and Standard errors")
            print("\n")
            print('a:',round(ans3[0],7),"±",round(perr3[0],7),'| ''b:', round(ans3[1],10),"±",round(perr3[1],3),'| ''c:', round(ans3[2],2),"±",round(perr3[2],3),'| ''Rd:', round(ans3[3],5),"±",round(perr3[3],3))

            pmax3= (1/(ans3[1]+2*((ans3[0]*ans3[2])**(1/2)))+ans3[3])
            s31=ans3[0]*ans3[2]
            serro31=((((perr3[0]/ans3[0])**2)+((perr3[2]/ans3[2])**2))*(s31**2))**(0.5)
            s311=s31**(0.5)
            erros311=(1/2)*(serro31/s31)*s311
            s312=2*s311
            erros312=((((erros311/s311)**2)+((0/ans3[2])**2))*(s312**2))**(0.5)
            erros313=((perr3[1]**2)+(erros312**2))**(0.5)
            s314=1/(s312+ans3[1])
            erros314=((((erros312/s312)**2)+((0/ans3[2])**2))*(s314**2))**(0.5)
            s315=s314+ans3[3]
            erros315=((perr3[3]**2)+(erros314**2))**(0.5)

            print("Pmax:", round(pmax3,2),"±",round(erros315,2))
            alfa=1/ans3[2]
            erroalfa3=((((perr3[2]/ans3[2])**2)+((0/ans3[2])**2))*(alfa**2))**(0.5)
            print("α:", round(alfa,2),"±",round(erroalfa3,2))

            ept3=((erros315**2)+(perr3[3]**2))**(0.5)
            pt3=pmax3+ans3[3]
            ek3= (pmax3)/alfa
            erroek3=(ept3/pt3)*(ek3)*(1/2)
            print("Ek:", round(ek3,2),"±",round(erroek3,2))
            ec3=-(ans3[3]/alfa)
            erroec3=(((((perr3[3]/ans3[3])**2))+((erroalfa3/alfa)**2))*(ec3**2))**(1/2)
            print("Ec:", round(ec3,2),"±",round(erroec3,2))
            bb3=(ans3[1])/(((ans3[0]*ans3[2])**(1/2)))
            errobbpt1=((((perr3[0]/ans3[0])**2)+((perr3[2]/ans3[2])**2))*((ans3[0]*ans3[2])**2))**(1/2)
            errobbpt2=errobbpt1*(1/2)*(((ans3[0]*ans3[2])**(1/2)))
            errobb=(erroalfa3/10)
            print("β:",round(bb3,2),"±",round(errobb,7))
            
            r23=r2_score(xdata,ftr3, sample_weight=None, multioutput='uniform_average')
            r2adj3=1-(1-r23)*(n-1)/(n-(Pj3-1) )
            rss3=((xdata-ftr3)**2).sum()
            print("R squared:", round(r23,3))
            print("Ajusted R squared:", round(r2adj3,3))
            mse3 = mean_squared_error(xdata, ftr3)
            aic3 = calculate_aic(n, rss3, Pj3)
            aiccc3=aic3+2*(Pj3*(Pj3)/(n-(Pj3)-1))
            
            sxy3=((((ftr3-xdata)**2).sum())/(n-4))**(0.5)
            print("Sxy:", round(sxy3,2))
            print("AICc:", round(aiccc3,2))
            
            rt=(ans[2])
            rt1=(ans1[2])
            rt3=(ans3[3])
            alfat=(ans[1])
            alfat1=(ans1[1])
            alfat3=(alfa)

            plt.figure(figsize=(6, 4))
            plt.grid(True)

            plt.plot(y,func_poli_inib(y,fit_a3,fit_b3,fit_c3,fit_r3), label='P=(E/((a*E**2)+(b*E+c)))+Rd') 
            plt.legend()
            plt.xlabel(ly)
            plt.ylabel(lx)
            plt.scatter(ydata,xdata)
            plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
            plt.savefig(f"/content/"+ "Rational.function.png")
            print("The program will continue to process after the graph is closed, it was saved in the same folder as the data file")
            plt.show()
            
            
            print(""                    '\n')

            
            
            
            



                    
      
            
            df2 = pd.DataFrame({'Model :':'tangent hyperbolic (P=Pmax*(tanh(alfa*E/Pmax)+Rd)' ,
                                'Pmax':round(ans[0],2) ,
                                'SE-1': (round(perr[0],2)),
                                'alfa':round(alfat,2) ,
                                'SE-2': (round(perr[1],2)),
                                'Ek':round(ek,2),
                                'SE-3': (round(erroek,2)),
                                'Ec':round(ec,2),
                                'SE-4': (round(erroec,2)),
                                'beta': ("-"),
                                'SE-5': ("-"),
                                'Rd': round(rt,2),
                                'SE-6': (round(perr[2],2)),
                                'R squared':round(r2,3),
                                'Ajusted R squared':round(r2adj,3),
                                'Sxy':round(sxy,2),
                                'AICc':round(aiccc,2),
                                },index= ["data"])
            df3 = pd.DataFrame({'Model :':'exponential (P=Pmax*(1-e^(-alfa*E/Pmax))+Rd)' ,
                                'Pmax':round(ans1[0],2) ,
                                'SE-1': (round(perr1[0],2)),
                                'alfa':round(alfat1,2) ,

                                'SE-2': (round(perr1[1],2)),
                                'Ek':round(ek1,2),
                                'SE-3': (round(erroek1,2)),
                                'Ec':round(ec1,2),
                                'SE-4': (round(erroec1,2)),
                                'beta': ("-"),
                                'SE-5': ("-"),
                                'Rd': round(rt1,2),
                                'SE-6': (round(perr1[2],2)),
                                'R squared':round(r21,3),
                                'Ajusted R squared':round(r2adj1,3),
                                'Sxy':round(sxy1,2),
                                'AICc':round(aiccc1,2),
                                },index= ["data"])
            df5 = pd.DataFrame({'Model :':'rational (P=(E/((a*E^2))+(b*E+c)))+Rd)'  ,
                                'Pmax':round(pmax3,2) ,
                                'SE-1': (round(erros315,2)),
                                'alfa':round(alfat3,2) ,
                                'SE-2': (round(erroalfa3,2)),
                                'Ek':round(ek3,2),
                                'SE-3': (round(erroek3,2)),
                                'Ec':round(ec3,2),
                                'SE-4': (round(erroec3,2)),
                                'beta':(round(bb3,7)),
                                'SE-5': (round(errobb,7)),
                                'Rd': round(rt3,2),
                                'SE-6': (round(perr3[3],2)),
                                'R squared': (round(r23,3)),
                                
                                'Ajusted R squared':round(r2adj3,3),
                                'Sxy':round(sxy3,2),
                                'AICc':round(aiccc3,2),
                                  },index= ["data"])
            frames=[df2,df3,df5]
            results = pd.concat(frames)
            results.to_csv(f"/content/" + "results.csv",sep=',')
            print("File containing the results was created in your folder content at google colab")
            print(""                     '\n')
            print(""                     '\n')
    else:
            print("Results:" )
            func_exp_inibi=print('Exponential function for cases of inhibition:'),print('P=(Ps*((1-e^(-α*E/Ps))*e^(-ß*E/Ps)))+Rd'),print('Ps='+str(round(fit_p2,2))+'*'+'((1-e^('+ '-' + str(round(fit_a2,3))+'*E/'+str(round(fit_p2,2))+'))*e^(-'+ str(round(fit_b2,5)) +'*E/'+ str(round(fit_p2,2))+'))'+ str(round(fit_r1,2)))
            print("\n")
            print("Data means and Standard errors")
            print("\n")                
            print('Ps:',round(ans2[0],2),"±",round(perr2[0],3),'| ''α:', round(ans2[1],4),"±",round(perr2[1],3),'| ''ß:', round(ans2[2],7),"±",round(perr2[2],3),'| ''Rd:', round(ans2[3],2),"±",round(perr2[3],3))
            


            pmax2= (ans2[0])*(ans2[1]/(ans2[1]+ans2[2]))*((ans2[2]/(ans2[1]+ans2[2]))**(ans2[2]/ans2[1]))+ans2[3]

            somaerro=ans2[1]+ans2[2]
            errosoma3=(((perr2[1])**2) + ((perr2[2])**2))**(0.5)
            somaa32 = ans2[1]/somaerro
            errosoma32=((((somaerro/errosoma3)**2)+((perr2[1]/ans2[1])**2))*(somaa32**2))**(0.5)
            soma322=ans2[2]/somaerro
            errosoma322=((((somaerro/errosoma3)**2)+((perr2[2]/ans2[2])**2))*(soma322**2))**(0.5)
            expoente1=(ans2[2]/ans2[1])
            erroexpoente1=((((perr2[1]/ans2[1])**2)+((perr2[2]/ans2[2])**2))*(expoente1**2))**(0.5)
            expoente2=soma322**expoente1
            erroexpoente2=(expoente1*(errosoma322/soma322))*expoente2
            multi1=somaa32*expoente2
            erromulti1=((((erroexpoente2/expoente2)**2)+((errosoma32/somaa32)**2))*(multi1**2))**(0.5)
            multi2=multi1*ans2[0]
            erromulti2=((((perr2[0]/ans2[0])**2)+((erromulti1/multi1)**2))*(multi2**2))**(0.5)
            somaf=multi2+ans2[3]
            errosomaf=(erromulti2+perr2[3])**(0.5)
            print("Pmax:", round(pmax2,2),"±",round(errosomaf,2))
            print('α:', round(ans2[1],4),"±",round(perr2[1],3))
            print('ß:', round(ans2[2],7),"±",round(perr2[2],3))
            
            r22=r2_score(xdata,ftr2, sample_weight=None, multioutput='uniform_average')
            r2adj2=1-((1-r22)*(n-1)/(n-(Pj2-1)))
            rss2=((xdata-ftr2)**2).sum()
            ept2=((errosomaf**2)+(perr2[3]**2))**(0.5)
            pt2=pmax2+ans2[3]
            ek2= (pmax2)/ans2[1]
            erroek2=(ept2/pt2)*(ek2)*(1/2)
            print("Ek:", round(ek2,2),"±",round(erroek2,2))
            ec2=-(ans2[3]/ans2[1])
            erroec2=(((((perr2[3]/ans2[3])**2))+((perr2[1]/ans2[1])**2))*(ec2**2))**(1/2)
            print("Ec:", round(ec2,2),"±",round(erroec2,2))
            print("R squared:", round(r22,3))
            print("Ajusted R squared:", round(r2adj2,3))
            mse2 = mean_squared_error(xdata, ftr2)
            aic2 = calculate_aic(n, rss2, Pj2)
            aiccc2=aic2+2*(Pj2*(Pj2)/(n-Pj2-1))
            
            sxy2=((((ftr2-xdata)**2).sum())/(n-4))**(0.5)
            print("Sxy:", round(sxy2,2))
            print("AICc:", round(aiccc2,2))






            plt.figure(figsize=(6, 4))
            plt.grid(True)

            plt.plot(y,func_exp_inib(y,fit_p2,fit_a2,fit_b2,fit_r2),label = 'P=Ps*(1-np.exp(-α*E/P)*np.exp(-ß*E/P))+Rd') 
            plt.legend()
            plt.xlabel(ly)
            plt.ylabel(lx)
            plt.scatter(ydata,xdata)
            plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
            plt.savefig(f"/content/"+ "exponential.double.function.png")
            print("\n")
            plt.show()
            

            print(""                     '\n')

            print("Results:" )
            func_poli_inibi=print('Rational  Function:'),print('P=(E/((a*E^2))+(b*E+c)))+Rd'),print('Ps= (E/(('+str(round(fit_a3,7))+'*E²)+('+ str(round(fit_b3,7))+'*E+'+ str(round(fit_c3,2)) +'))+('+ str(round(fit_r3,2))+')')
            print("\n")
            print("Data means and Standard errors")
            print("\n")
            print('a:',round(ans3[0],7),"±",round(perr3[0],7),'| ''b:', round(ans3[1],10),"±",round(perr3[1],3),'| ''c:', round(ans3[2],2),"±",round(perr3[2],3),'| ''Rd:', round(ans3[3],5),"±",round(perr3[3],3))
            
            
            pmax3= (1/(ans3[1]+2*((ans3[0]*ans3[2])**(1/2)))+ans3[3])
            s31=ans3[0]*ans3[2]
            serro31=((((perr3[0]/ans3[0])**2)+((perr3[2]/ans3[2])**2))*(s31**2))**(0.5)
            s311=s31**(0.5)
            erros311=(1/2)*(serro31/s31)*s311
            s312=2*s311
            erros312=((((erros311/s311)**2)+((0/ans3[2])**2))*(s312**2))**(0.5)
            erros313=((perr3[1]**2)+(erros312**2))**(0.5)
            s314=1/(s312+ans3[1])
            erros314=((((erros312/s312)**2)+((0/ans3[2])**2))*(s314**2))**(0.5)
            s315=s314+ans3[3]
            erros315=((perr3[3]**2)+(erros314**2))**(0.5)
            print("Pmax:", round(pmax3,2),"±",round(erros315,2))
            alfa=1/ans3[2]
            erroalfa3=((((perr3[2]/ans3[2])**2)+((0/ans3[2])**2))*(alfa**2))**(0.5)
            print("α:", round(alfa,2),"±",round(erroalfa3,2))
            ept3=((erros315**2)+(perr3[3]**2))**(0.5)
            pt3=pmax3+ans3[3]
            ek3= (pmax3)/alfa
            erroek3=(ept3/pt3)*(ek3)*(1/2)
            print("Ek:", round(ek3,2),"±",round(erroek3,2))
            ec3=-(ans3[3]/alfa)
            erroec3=(((((perr3[3]/ans3[3])**2))+((erroalfa3/alfa)**2))*(ec3**2))**(1/2)
            print("Ec:", round(ec3,2),"±",round(erroec3,2))
            bb3=(ans3[1])/(((ans3[0]*ans3[2])**(1/2)))
            errobbpt1=((((perr3[0]/ans3[0])**2)+(perr3[2]/ans3[2])**2)*((ans3[0]*ans3[2])**2))**(1/2)
            errobbpt2=errobbpt1*(1/2)*(((ans3[0]*ans3[2])**1/2))
            errobb=erroalfa3/10
            print("β:",round(bb3,2),"±",round(errobb,7))
            
            r23=r2_score(xdata,ftr3, sample_weight=None, multioutput='uniform_average')
            r2adj3=1-(1-r23)*(n-1)/(n-(Pj3-1) )
            rss3=((xdata-ftr3)**2).sum()
            print("R squared:", round(r23,3))
            print("Ajusted R squared:", round(r2adj3,3))
            mse3 = mean_squared_error(xdata, ftr3)
            aic3 = calculate_aic(n, rss3, Pj3)
            aiccc3=aic3+2*(Pj3*(Pj3)/(n-(Pj3)-1))
            
            sxy3=((((ftr3-xdata)**2).sum())/(n-4))**(0.5)
            print("Sxy:", round(sxy3,2))
            print("AICc:", round(aiccc3,2))

            plt.figure(figsize=(6, 4))
            plt.grid(True)

            plt.plot(y,func_poli_inib(y,fit_a3,fit_b3,fit_c3,fit_r3), label='P=(E/((a*E**2)+(b*E+c)))+Rd') 
            plt.legend()
            plt.xlabel(ly)
            plt.ylabel(lx)
            plt.scatter(ydata,xdata)
            plt.errorbar(ydata, xdata, yerr=xstd, fmt='o')
            plt.savefig(f"/content/"+"rational function.png")
            print("\n")
            plt.show()
            
            
            
            rt2=(ans2[3])
            rt3=(ans3[3])
            
            alfat2=(ans2[1])
            alfat3=(alfa)
            pmaxt2=(ans2[0])
            epmaxt2=(perr2[0])
            
            
            df4 = pd.DataFrame({'Model :':'double exponential (P=(Ps*((1-e^(-alfa*E/Ps))*e^(-ß*E/Ps)))+Rd)'  ,
                                        'Pmax':round(pmax2,2) ,
                                        'SE-1': (round(errosomaf,2)),
                                        'alfa':round(alfat2,2) ,
                                        'SE-2': (round(perr2[1],2)),
                                        'Ek':round(ek2,2),
                                        'SE-3': (round(erroek2,2)),
                                        'Ec':round(ec2,2),
                                        'SE-4': (round(erroec2,2)),
                                        'beta':(round(ans2[2],7)),
                                        'SE-5': (round(perr2[2],7)),
                                        'Rd':round(rt2,2),
                                        'SE-6': (round(perr2[3],2)),
                                        'R squared':round(r22,3),
                                        'Ajusted R squared':round(r2adj2,3),
                                        'Sxy':round(sxy2,2),
                                        'AICc':round(aiccc2,2),
                  },index= ["data"])
            df5 = pd.DataFrame({'Model :':'rational (P=(E/((a*E^2))+(b*E+c)))+Rd)'  ,
                                'Pmax':round(pmax3,2) ,
                                'SE-1': (round(erros315,2)),
                                'alfa':round(alfat3,2) ,
                                'SE-2': (round(erroalfa3,2)),
                                'Ek':round(ek3,2),
                                'SE-3': (round(erroek3,2)),
                                'Ec':round(ec3,2),
                                'SE-4': (round(erroec3,2)),
                                'beta':(round(bb3,2)),
                                'SE-5': (round(errobb,2)),
                                'Rd': round(rt3,2),
                                'SE-6': (round(perr3[3],2)),
                                'R squared':round(r23,3),
                                'Ajusted R squared':round(r2adj3,3),
                                'Sxy':round(sxy3,2),
                                'AICc':round(aiccc3,2),
                  },index= ["data"])




  


    

            frames=[df4,df5]
            results = pd.concat(frames)
            results.to_csv(f"/content/" + "results.csv",sep=',')
            print("File containing the results was created in your folder content at google colab")
            print(""                    '\n')
            print(""                    '\n')

In [ ]:
from google.colab import output
output.clear()

mainMenu()





 Software for fitting Photosynthetic Light-Response Curve in aquatic environment 
                                   by
                      Arthur Lins  &  Paulo Lins
Universidade Federal Fluminense (UFF), arthurlins@id.uff.br 
Secretaria de Estado de Educação do Rio de Janeiro (SEDUC),  plins@prof.educacao.rj.gov.br


 1.Import data from a spreadsheet  (.csv separated by commas).
 2.Enter data
 3.Reference and Photosynthetic Parameters 
 4.Results
Choose the option:
